In [1]:
import os

In [2]:
%pwd

'e:\\Retail-Analysis-with-Walmart\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Retail-Analysis-with-Walmart'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path


In [8]:
from real_walmart.constants import *
from real_walmart.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config


In [10]:
import numpy as np
import os
import pandas as pd
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from real_walmart.logging import logger

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        try:
            logger.info("Starting model evaluation")

            # Load test data
            test_df = pd.read_csv(self.config.test_data_path)

            X_test = test_df.drop(columns=["Weekly_Sales"])
            y_test = test_df["Weekly_Sales"]

            # Load trained model
            model = joblib.load(self.config.model_path)

            # Predictions
            y_pred = model.predict(X_test)

            # Metrics (version-safe)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            metrics = {
                "rmse": rmse,
                "mae": mae,
                "r2_score": r2
            }

            # Save metrics
            os.makedirs(self.config.root_dir, exist_ok=True)
            pd.DataFrame([metrics]).to_csv(
                self.config.metric_file_name, index=False
            )

            logger.info(f"Model evaluation completed: {metrics}")

        except Exception as e:
            raise e

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()

    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()

except Exception as e:
    raise e

[2025-12-21 08:39:44,169: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-12-21 08:39:44,205: INFO: common: YAML file loaded successfully: params.yaml]
[2025-12-21 08:39:44,210: INFO: common: Created directory at: artifacts]
[2025-12-21 08:39:44,215: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-12-21 08:39:44,234: INFO: 995592838: Starting model evaluation]


[2025-12-21 08:39:45,968: INFO: 995592838: Model evaluation completed: {'rmse': np.float64(145552.02836361964), 'mae': 76055.08392859358, 'r2_score': 0.9342384206635401}]
